In [414]:
import os
import dotenv
import requests
import numpy as np
import pandas as pd

In [415]:
# Load environment variables
project_dir = os.path.join(os.path.abspath(''), os.pardir)
dotenv_path = os.path.join(project_dir, '.env')

dotenv.load_dotenv(dotenv_path)

True

In [416]:
df = pd.read_csv(os.path.join(os.getenv('DATA_PATH'), 'data_cols_extended.csv'), low_memory=False, keep_default_na = True, na_values=['\\N', 0, 0.0, '0', '0.0'])

In [417]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87927 entries, 0 to 87926
Data columns (total 73 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Projekt                        87927 non-null  object 
 1   Campaign                       87927 non-null  object 
 2   Subcampaign                    87927 non-null  object 
 3   Paid / Organic                 87927 non-null  object 
 4   Campaign Name                  1 non-null      object 
 5   FB Ad Name                     0 non-null      float64
 6   Company                        87927 non-null  object 
 7   Channel                        87927 non-null  object 
 8   Subchannel                     87927 non-null  object 
 9   Content Type                   87927 non-null  object 
 10  DarkPublic                     87927 non-null  object 
 11  Content ID                     87927 non-null  object 
 12  share.id                       87899 non-null 

In [418]:
df_post_dates = pd.read_csv(os.path.join(os.getenv('DATA_EXT_PATH'), 'found_timestamps.csv'))
df_post_dates

,ID,Post Ts
0,urn:li:ugcPost:7080032059519373312,2023-06-29 05:59:43.870
1,urn:li:share:7049282071755694080,2023-04-05 09:30:15.391
2,urn:li:share:7057277321598132224,2023-04-27 11:00:31.505
3,urn:li:share:7021364524641394688,2023-01-18 07:35:53.486
4,urn:li:share:7089510852810166272,2023-07-25 09:45:04.695
...,...,...
4328,urn:li:share:7072474322124120064,2023-06-08 09:27:58.783
4329,urn:li:ugcPost:7069272909654974464,2023-05-30 13:26:43.492
4330,urn:li:share:7068926451479191553,2023-05-29 14:30:00.470
4331,urn:li:share:7032333183689842688,2023-02-17 14:01:25.618


In [419]:
df_cut = df.loc[:,
[
    'Content ID', 'share.id','Permalink',
    'Message Text', 'Date', 'Impressions Unpaid', 'Comments', 'Likes',
    'Shares', 'Other Engagements', 'Engagements', 'Image URL',
    'Post Date', 'Days since Post Date', 'fanpage_id', 'all_hashtags'
]]

In [420]:
df_cut = df_cut[~df_cut['share.id'].isna()]
df_cut = df_cut[~df_cut['Message Text'].isna()]
df_cut = df_cut[~df_cut['Impressions Unpaid'].isna()]

In [421]:
test1 = df_cut[df_cut['Message Text'].str.match(r"no text \(\d{2}.\d{2}.\d{2}\)")]['share.id'].unique()
# check if some message text can be replaced instead of dummy texts
df_cut[df_cut['share.id'].isin(test1) & (~df_cut['Message Text'].str.match(r"no text \(\d{2}.\d{2}.\d{2}\)"))]

,Content ID,share.id,Permalink,Message Text,Date,Impressions Unpaid,Comments,Likes,Shares,Other Engagements,Engagements,Image URL,Post Date,Days since Post Date,fanpage_id,all_hashtags


In [422]:
df_cut = df_cut[~df_cut['Message Text'].str.match(r"no text \(\d{2}.\d{2}.\d{2}\)")]

In [423]:
df_cut.loc[:, 'ID'] = df_cut['share.id']

In [424]:
df_cut.loc[:, 'Post Date'] = pd.to_datetime(df_cut['Post Date'])

In [425]:
df_cut.loc[:, 'Date'] = pd.to_datetime(df_cut['Date'])

In [426]:
df_cut.loc[:, 'has_any_image'] = np.where(df_cut['Image URL'].str.len() > 0, True, False)

In [427]:
df_cut.loc[:, 'hashtags'] = df_cut['all_hashtags'].str.replace('#', '').apply(lambda x: ','.join(filter(len, x.split())) if isinstance(x, str) else x)

In [428]:
df_cut.loc[:, 'has_any_hashtag'] = np.where(df_cut['hashtags'].str.len() > 0, True, False)

In [429]:
df_cut.loc[:, 'dow_posted'] = pd.to_datetime(df_cut['Post Date']).dt.day_name()

In [430]:
df_cut.loc[:, 'post_len'] = df_cut['Message Text'].str.len()

In [ ]:
# Use if provided timestamps not available // TOKEN needed
# from datetime import datetime
# from linkedin_api.clients.restli.client import RestliClient

# restli_client = RestliClient()
# posts = df_cut.ID.unique()
# zips = []
# created_ats = []
# split_by_size = lambda x, size: np.split( x, np.arange(size, len(x), size) )
# chunks = split_by_size(x=posts, size=100)
# for chunk in chunks:
#     resp = restli_client.batch_get(
#       resource_path="/posts",
#       ids=list(chunk),
#       access_token=os.getenv('API_LI_TOKEN'),
#       version_string="202304"
#     )
#     print(resp)
#     li_items = resp.results.items()
#     tss = [datetime.fromtimestamp(i[1].get('createdAt', 0)/1000) for i in li_items]
#     ids = [i[0] for i in li_items]
#     res = zip(ids, tss)
#     zips.append(res)
#     created_ats.extend(list(res))
# df_post_dates = pd.DataFrame(created_ats, columns =['ID', 'Post Ts'])

In [431]:
df_cut_with_ts = pd.merge(
    df_cut,
    df_post_dates,
    on = ['ID'],
    how = 'left'
)

In [432]:
df_cut_with_ts.loc[df_cut_with_ts['Post Ts'].isna(), 'Post Ts'] = pd.to_datetime(df_cut_with_ts['Post Date'])

In [433]:
df_cut_with_ts.loc[:, 'hm_posted'] = pd.to_datetime(df_cut_with_ts['Post Ts'], errors='coerce').dt.strftime('%H:%M')

In [434]:
df_cut_with_ts.loc[:, 'hour_posted'] = pd.to_datetime(df_cut_with_ts['Post Ts'], errors='coerce').dt.strftime('%H').astype('int')

In [435]:
df_cut_with_ts.loc[df_cut_with_ts['Days since Post Date'].isna(), 'Days since Post Date'] = 0

In [436]:
df_cut_with_ts.head()

,Content ID,share.id,Permalink,Message Text,Date,Impressions Unpaid,Comments,Likes,Shares,Other Engagements,...,all_hashtags,ID,has_any_image,hashtags,has_any_hashtag,dow_posted,post_len,Post Ts,hm_posted,hour_posted
0,urn:li:activity:7049687222605754368,urn:li:share:7049687219682299905,https://www.linkedin.com/feed/update/urn:li:ac...,2023 – the year of #ChatGPT and Co. 🤯 Will gen...,2023-04-18 00:00:00,186.0,NaN,NaN,NaN,1.0,...,#AI #ArtificialIntelligence #ChatGPT #Generati...,urn:li:share:7049687219682299905,False,"AI,ArtificialIntelligence,ChatGPT,GenerativeAI...",True,Thursday,730,2023-04-06 12:20:10.694,12:20,12
1,urn:li:activity:7079078882049478656,urn:li:share:7079078881395167233,https://www.linkedin.com/feed/update/urn:li:ac...,Smart Mechatronix: the perfect ready-to-instal...,2023-07-03 00:00:00,7.0,NaN,NaN,NaN,NaN,...,#Mechatronix,urn:li:share:7079078881395167233,False,Mechatronix,True,Monday,284,2023-06-26 14:52:08.375,14:52,14
2,urn:li:activity:7054408338465300481,urn:li:share:7054408337148325888,https://www.linkedin.com/feed/update/urn:li:ac...,"We love oranges so much, we want to extract as...",2023-04-30 00:00:00,1191.0,1.0,14.0,NaN,32.0,...,#foodindustry #foodingredients #orange #orange...,urn:li:share:7054408337148325888,False,"foodindustry,foodingredients,orange,orangeflav...",True,Wednesday,899,2023-04-19 13:00:12.335,13:00,13
3,urn:li:ugcPost:7069950389063667712,urn:li:ugcPost:7069950389063667712,https://www.linkedin.com/feed/update/urn:li:ug...,“Inclusion starts with I” – @[Nadine Ilgenstei...,2023-06-18 00:00:00,788.0,NaN,4.0,NaN,73.0,...,#Diversity #Inclusion #WeAreSchaeffler,urn:li:ugcPost:7069950389063667712,False,"Diversity,Inclusion,WeAreSchaeffler",True,Monday,803,2023-06-01 10:18:46.279,10:18,10
4,urn:li:activity:7057277322424410112,urn:li:share:7057277321598132224,https://www.linkedin.com/feed/update/urn:li:ac...,MediaMarktSaturn geht den nächsten digitalen S...,2023-05-19 00:00:00,195.0,NaN,1.0,NaN,3.0,...,#experiencechampion #Retail #Service #Sustaina...,urn:li:share:7057277321598132224,False,"experiencechampion,Retail,Service,Sustainabili...",True,Thursday,773,2023-04-27 11:00:31.505,11:00,11


In [437]:
df_cut_final = df_cut_with_ts.loc[:, 
[
    'Message Text', 'Date',
    'Impressions Unpaid', 'Comments', 'Likes', 'Shares',
    'Other Engagements', 'Engagements',
    'Days since Post Date', 'fanpage_id',
    'ID', 'has_any_image', 'hashtags', 'has_any_hashtag', 'dow_posted',
    'post_len', 'Post Ts', 'hm_posted', 'hour_posted'
]].rename(columns={
    'Impressions Unpaid': 'Impressions',
    'Days since Post Date': 'days_since_posted',
    'Post Ts': 'timestamp_posted'
})

In [438]:
df_cut_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85904 entries, 0 to 85903
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Message Text       85904 non-null  object 
 1   Date               85904 non-null  object 
 2   Impressions        85904 non-null  float64
 3   Comments           3817 non-null   float64
 4   Likes              31898 non-null  float64
 5   Shares             8472 non-null   float64
 6   Other Engagements  34749 non-null  float64
 7   Engagements        41702 non-null  float64
 8   days_since_posted  85904 non-null  float64
 9   fanpage_id         85904 non-null  float64
 10  ID                 85904 non-null  object 
 11  has_any_image      85904 non-null  bool   
 12  hashtags           74792 non-null  object 
 13  has_any_hashtag    85904 non-null  bool   
 14  dow_posted         85904 non-null  object 
 15  post_len           85904 non-null  int64  
 16  timestamp_posted   859

In [439]:
df_cut_final.head()

,Message Text,Date,Impressions,Comments,Likes,Shares,Other Engagements,Engagements,days_since_posted,fanpage_id,ID,has_any_image,hashtags,has_any_hashtag,dow_posted,post_len,timestamp_posted,hm_posted,hour_posted
0,2023 – the year of #ChatGPT and Co. 🤯 Will gen...,2023-04-18 00:00:00,186.0,NaN,NaN,NaN,1.0,1.0,12.0,157764.0,urn:li:share:7049687219682299905,False,"AI,ArtificialIntelligence,ChatGPT,GenerativeAI...",True,Thursday,730,2023-04-06 12:20:10.694,12:20,12
1,Smart Mechatronix: the perfect ready-to-instal...,2023-07-03 00:00:00,7.0,NaN,NaN,NaN,NaN,NaN,7.0,56825.0,urn:li:share:7079078881395167233,False,Mechatronix,True,Monday,284,2023-06-26 14:52:08.375,14:52,14
2,"We love oranges so much, we want to extract as...",2023-04-30 00:00:00,1191.0,1.0,14.0,NaN,32.0,47.0,11.0,2269.0,urn:li:share:7054408337148325888,False,"foodindustry,foodingredients,orange,orangeflav...",True,Wednesday,899,2023-04-19 13:00:12.335,13:00,13
3,“Inclusion starts with I” – @[Nadine Ilgenstei...,2023-06-18 00:00:00,788.0,NaN,4.0,NaN,73.0,77.0,6.0,118.0,urn:li:ugcPost:7069950389063667712,False,"Diversity,Inclusion,WeAreSchaeffler",True,Monday,803,2023-06-01 10:18:46.279,10:18,10
4,MediaMarktSaturn geht den nächsten digitalen S...,2023-05-19 00:00:00,195.0,NaN,1.0,NaN,3.0,4.0,22.0,155056.0,urn:li:share:7057277321598132224,False,"experiencechampion,Retail,Service,Sustainabili...",True,Thursday,773,2023-04-27 11:00:31.505,11:00,11


In [440]:
df_cut_final.to_csv(os.path.join(os.getenv('SAVE_INTERIM_FILES_PATH'), 'data_interim_new_features.csv'), index=False, encoding='utf-8')